Install dependency tfx

In [3]:
!pip install tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
%cd /content/drive/MyDrive/MLOps_Proyek_Akhir

/content/drive/MyDrive/MLOps_Proyek_Akhir


Dilakukan import dan juga ditentukan path untuk pipeline, input, dan juga output

In [16]:
import os
import sys
from typing import Text

from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

PIPELINE_NAME = "femi_nabila_2zI8-pipeline"

# pipeline inputs
DATA_ROOT = "/content/drive/MyDrive/MLOps_Proyek_Akhir/data"
TRANSFORM_MODULE_FILE = "/content/drive/MyDrive/MLOps_Proyek_Akhir/modules/lovehate_transform.py"
TRAINER_MODULE_FILE = "/content/drive/MyDrive/MLOps_Proyek_Akhir/modules/lovehate_trainer.py"
# requirement_file = os.path.join(root, "requirements.txt")

# pipeline outputs
OUTPUT_BASE = "/content/output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model_dir')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")


Dibuat function init_local_pipeline untuk menyatukan seluruh TFX component.

In [17]:
def init_local_pipeline(components, pipeline_root: Text) -> pipeline.Pipeline:

    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing",
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        "--direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        beam_pipeline_args=beam_args
    )


Run init_components untuk membuat TFX components, init_local_pipeline untuk disatukan pada pipeline, dan juga menjalankan pipeline tersebut dengan Apache Beam

In [18]:
if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)

    from modules.components import init_components

    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        training_steps=5000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir,
    )

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Pipeline root set to: /content/output/femi_nabila_2zI8-pipeline
INFO:absl:Generating ephemeral wheel package for '/content/drive/MyDrive/MLOps_Proyek_Akhir/modules/lovehate_transform.py' (including modules: ['lovehate_trainer', 'lovehate_transform', 'components']).
INFO:absl:User module package has hash fingerprint version d9a0bf278b38ea732c965106a1d3890c74cc158202ced7dacf61e1d54a02ede5.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmpypvwecpe/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpwkbu7s5z', '--dist-dir', '/tmp/tmpsbpi5zg_']
INFO:absl:Successfully built user code wheel distribution at '/content/output/femi_nabila_2zI8-pipeline/_wheels/tfx_user_code_Transform-0.0+d9a0bf278b38ea732c965106a1d3890c74cc158202ced7dacf61e1d54a02ede5-py3-none-any.whl'; target use

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sentence_xf (InputLayer)    [(None, 1)]               0         
                                                                 
 tf.reshape_2 (TFOpLambda)   (None,)                   0         
                                                                 
 text_vectorization_2 (Text  (None, 100)               0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_2  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_6 (Dense)             (None, 64)                1088


Epoch 1: val_binary_accuracy improved from -inf to 0.74457, saving model to /content/output/femi_nabila_2zI8-pipeline/Trainer/model/7/Format-Serving


INFO:absl:Function `_wrapped_model` contains input name(s) table_handle, 29135, resource with unsupported characters which will be renamed to model_2_text_vectorization_2_string_lookup_6_none_lookup_lookuptablefindv2_table_handle, model_2_embedding_embedding_lookup_29135, model_2_dense_8_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/output/femi_nabila_2zI8-pipeline/Trainer/model/7/Format-Serving/fingerprint.pb


1000/1000 [==============================] - 5s 4ms/step - loss: 0.4911 - binary_accuracy: 0.7173 - val_loss: 0.5808 - val_binary_accuracy: 0.7446


INFO:absl:Feature sentence has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature  has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Function `serve_tf_examples_fn` contains input name(s) table_handle, 30212, resource with unsupported characters which will be renamed to model_2_text_vectorization_2_string_lookup_6_hash_table_lookup_lookuptablefindv2_table_handle, model_2_embedding_embedding_lookup_30212, model_2_dense_8_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to /content/output/femi_nabila_2zI8-pipeline/Trainer/model/7/Format-Serving/fingerprint.pb
INFO:absl:Training complete. Model written to /content/output/femi_nabila_2zI8-pipeline/Trainer/model/7/Format-Serving. ModelRun written 

In [19]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/femi_nabila_2zI8-pipeline/ (stored 0%)
  adding: content/output/femi_nabila_2zI8-pipeline/metadata.sqlite (deflated 94%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/ (stored 0%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/pushed_model/ (stored 0%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/pushed_model/9/ (stored 0%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/pushed_model/9/fingerprint.pb (stored 0%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/pushed_model/9/saved_model.pb (deflated 86%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/pushed_model/9/keras_metadata.pb (deflated 85%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/pushed_model/9/variables/ (stored 0%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pusher/pushed_model/9/variables/variables.index (deflated 59%)
  adding: content/output/femi_nabila_2zI8-pipeline/Pushe